###  Loading the Boston housing dataset

In [1]:
from tensorflow.keras.datasets import boston_housing
from sklearn.preprocessing import normalize
from tensorflow.keras import layers, models
from sklearn.model_selection import KFold

import numpy as np 

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [3]:
print(train_data[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [4]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


We can see that we have 404 training samples and 102 test samples, each with 13 numerical features.

In [5]:
print(train_targets[0])

15.2


The targets are the median values of the houses at a location in 1000's of dollars.

### Preparing the data

In [6]:
normalize(train_data, axis=0) # Feature-wise normalization

array([[6.15626701e-03, 0.00000000e+00, 3.10983151e-02, ...,
        5.61526372e-02, 5.38018011e-02, 6.35435054e-02],
       [1.08742552e-04, 1.55659962e-01, 7.75547662e-03, ...,
        3.93068460e-02, 5.35957574e-02, 1.05566401e-02],
       [2.44669243e-02, 0.00000000e+00, 6.91498162e-02, ...,
        5.40134891e-02, 5.09036340e-02, 1.10658028e-02],
       ...,
       [1.73128932e-04, 6.60375595e-02, 2.31518169e-02, ...,
        4.51895032e-02, 4.91048185e-02, 2.65782931e-02],
       [1.07352925e-02, 0.00000000e+00, 7.48040553e-02, ...,
        3.93068460e-02, 3.55086465e-02, 5.35978606e-02],
       [7.18789766e-05, 1.13207245e-01, 1.11938653e-02, ...,
        4.17133876e-02, 5.10635890e-02, 1.48675509e-02]])

### Building the neural network

In [7]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1], )))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1)) # Output layer. Single unit and no activation
    
    model.compile(loss='mse', # Computes the mean squared error between labels and predictions.
                  optimizer='rmsprop',
                  metrics=['mae']) # Computes the mean absolute error between labels and predictions.
    return model

### K-fold validation

In [8]:
K = 4
k_fold_validation = KFold(n_splits=K)
k_fold_validation.get_n_splits(train_data) # Returns the number of splitting iterations in the cross-validator

4

In [9]:
EPOCHS = 500
all_scores = []
for train_index, valid_index in k_fold_validation.split(train_data):
    model = build_model()
    model.fit(train_data[train_index], train_targets[train_index],
              epochs=EPOCHS, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(train_data[valid_index], train_targets[valid_index],
                                      verbose=0)
    all_scores.append(val_mae) 

In [11]:
print(all_scores)

[3.213961362838745, 2.799401044845581, 3.2707102298736572, 3.460131883621216]


In [14]:
np.mean(all_scores)

3.1860511302948